<a href="https://colab.research.google.com/github/JoyciPM/ProjetoML2/blob/main/27_Maio_Etapa_IIipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Amazon Kindle Book Review for Sentiment Analysis**

https://www.kaggle.com/datasets/meetnagadia/amazon-kindle-book-review-for-sentiment-analysis

Os resultados mostram que, quando as pessoas escrevem revisões com sentimentos positivos, outros leitores acham essas revisões mais úteis. Mas a relação não é muito forte, o que significa que revisões positivas não são sempre as mais úteis. Existem outras coisas que também podem fazer uma revisão ser útil. Por exemplo, quanto detalhe a pessoa deu, se a revisão é fácil de entender, e assim por diante. Então, sentimentos positivos ajudam, mas não são a única coisa que importa.

No projeto, usamos PLN para analisar os sentimentos nas revisões. Isso significa que usamos ferramentas para entender se o texto das revisões era positivo, negativo ou neutro. A análise de sentimentos nos ajudou a ver que revisões com sentimentos positivos tendem a ser consideradas mais úteis pelos leitores.

# Resumo das etapas

*   Pré-processamento de Texto: Remoção de números, pontuação, conversão para minúsculas, remoção de stopwords e lematização.
*   Análise de Sentimento: Cálculo da polaridade dos sentimentos usando TextBlob.
*   Vetorização TF-IDF: Transformação dos textos em vetores de características usando TF-IDF.
*   Elbow Method: Identificação do número ideal de clusters.
*   K-Means Clustering: Agrupamento dos reviews em clusters.
*   Redução de Dimensionalidade: Uso de PCA para visualização dos clusters.
*   Análise dos Clusters: Exibição das primeiras 5 reviews de cada cluster e a média do sentimento.

# **Análise com PLN e agrupamento**

In [1]:
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import Textblob: tem o propósito de Processamento de Linguagem Natural e análise de sentimento. Segundo a documentação, a TextBlob é uma biblioteca Python voltada para o processamento de dados textuais, o principal objeto de estudo do NLP. Além disso, ela tem a missão de simplificar alguns processos importantes, usando - por baixo dos panos - as bibliotecas NLTK e Pattern.

In [2]:
# Baixar recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Carregar dataset
df = pd.read_csv('all_kindle_review .csv')

In [5]:
import pandas as pd

# Carregar dataset
df = pd.read_csv('all_kindle_review .csv')

# Ver o nome das colunas
print("Nomes das colunas:")
print(df.columns)

# Ver o tipo dos dados
print("\nTipos dos dados:")
print(df.dtypes)

Nomes das colunas:
Index(['Unnamed: 0.1', 'Unnamed: 0', 'asin', 'helpful', 'rating', 'reviewText',
       'reviewTime', 'reviewerID', 'reviewerName', 'summary',
       'unixReviewTime'],
      dtype='object')

Tipos dos dados:
Unnamed: 0.1       int64
Unnamed: 0         int64
asin              object
helpful           object
rating             int64
reviewText        object
reviewTime        object
reviewerID        object
reviewerName      object
summary           object
unixReviewTime     int64
dtype: object


In [8]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,jace rankin may short he nothing mess man haul...,"09 2, 2010",A3HHXRELK8BHQG,Ridley,entertaining average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,great short read didnt want put read one sitti...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,terrific menage scene,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,ill start saying first four book wasnt expecti...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,snapdragon alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,aggie angela lansbury carry pocketbook instead...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,expect type book library pleased find price right,"12 31, 2012",A3C9V987IQHOQD,Rjostler,book,1356912000


In [6]:
# Limpeza dos textos
def preprocess_text(text):
    text = re.sub(r'\d+', '', text)  # Remove números
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    text = text.lower()  # Converte para minúsculas
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stop words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lematização
    return ' '.join(tokens)

df['reviewText'] = df['reviewText'].fillna('').astype(str).apply(preprocess_text)
df['summary'] = df['summary'].fillna('').astype(str).apply(preprocess_text)

In [ ]:
# Análise de Sentimento
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Verificar se a coluna reviewText_sentiment já existe
if 'reviewText_sentiment' not in df.columns:
    df['reviewText_sentiment'] = df['reviewText'].apply(get_sentiment)
if 'summary_sentiment' not in df.columns:
    df['summary_sentiment'] = df['summary'].apply(get_sentiment)

In [ ]:
# Vetorização TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['reviewText'])

**Método K-Means - Método do cotovelo**

Então é preciso achar um equilíbrio em que as observações que formam cada agrupamento sejam o mais homogêneas possível e que os agrupamentos formados sejam o mais diferentes um dos outros.


In [ ]:
# Escolha do número de clusters usando o "Elbow Method"
inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(K, inertia, 'bx-')
plt.xlabel('Número de clusters')
plt.ylabel('Inércia')
plt.title('Elbow Method para encontrar o número ótimo de clusters')
plt.show()

In [ ]:
# Aplicar K-Means
num_clusters = 5  # Ajustar conforme necessário
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Redução de dimensionalidade para visualização
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X.toarray())

In [ ]:
# Plotar os clusters
plt.figure(figsize=(10, 8))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=df['cluster'], cmap='viridis')
plt.xlabel('Componente PCA 1')
plt.ylabel('Componente PCA 2')
plt.title('Clusters de Reviews')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
# Análise dos clusters
for cluster in range(num_clusters):
    print(f"Cluster {cluster}")
    cluster_reviews = df[df['cluster'] == cluster]
    print(cluster_reviews['reviewText'].head(5))  # Mostrar as primeiras 5 revisões do cluster
    print(f"Média do Sentimento: {cluster_reviews['reviewText_sentiment'].mean():.4f}")
    print("\n")